## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ponta Do Sol,PT,32.6667,-17.1000,69.71,66,65,7.96,broken clouds
1,1,Vanimo,PG,-2.6741,141.3028,84.61,73,60,7.16,broken clouds
2,2,Itaueira,BR,-7.6033,-43.0256,78.89,43,69,5.06,broken clouds
3,3,Srandakan,ID,-7.9386,110.2506,84.79,73,99,10.74,overcast clouds
4,4,Dikson,RU,73.5069,80.5464,9.10,86,100,6.04,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vanimo,PG,-2.6741,141.3028,84.61,73,60,7.16,broken clouds
2,2,Itaueira,BR,-7.6033,-43.0256,78.89,43,69,5.06,broken clouds
3,3,Srandakan,ID,-7.9386,110.2506,84.79,73,99,10.74,overcast clouds
5,5,Rikitea,PF,-23.1203,-134.9692,75.52,81,45,16.73,scattered clouds
9,9,Touros,BR,-5.1989,-35.4608,77.90,79,52,12.75,broken clouds
10,10,Hirara,JP,24.8000,125.2833,79.07,65,40,8.05,scattered clouds
13,13,Pedernales,DO,18.0384,-71.7440,79.54,75,68,5.35,broken clouds
21,21,Kapaa,US,22.0752,-159.3190,80.58,93,75,12.66,light rain
26,26,Tosham,IN,28.8833,75.9167,75.60,34,0,2.68,clear sky
34,34,Acarau,BR,-2.8856,-40.1200,78.57,81,52,10.29,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                177
City                   177
Country                177
Lat                    177
Lng                    177
Max Temp               177
Humidity               177
Cloudiness             177
Wind Speed             177
Current Description    177
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#N/A as all rows have data

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vanimo,PG,84.61,broken clouds,-2.6741,141.3028,
2,Itaueira,BR,78.89,broken clouds,-7.6033,-43.0256,
3,Srandakan,ID,84.79,overcast clouds,-7.9386,110.2506,
5,Rikitea,PF,75.52,scattered clouds,-23.1203,-134.9692,
9,Touros,BR,77.90,broken clouds,-5.1989,-35.4608,
10,Hirara,JP,79.07,scattered clouds,24.8000,125.2833,
13,Pedernales,DO,79.54,broken clouds,18.0384,-71.7440,
21,Kapaa,US,80.58,light rain,22.0752,-159.3190,
26,Tosham,IN,75.60,clear sky,28.8833,75.9167,
34,Acarau,BR,78.57,broken clouds,-2.8856,-40.1200,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   177
Country                177
Max Temp               177
Current Description    177
Lat                    177
Lng                    177
Hotel Name             177
dtype: int64

In [13]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vanimo,PG,84.61,broken clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
2,Itaueira,BR,78.89,broken clouds,-7.6033,-43.0256,Pousada e Restaurante Cheiro e Sabor
3,Srandakan,ID,84.79,overcast clouds,-7.9386,110.2506,Puri Brata
5,Rikitea,PF,75.52,scattered clouds,-23.1203,-134.9692,People ThankYou
9,Touros,BR,77.90,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
10,Hirara,JP,79.07,scattered clouds,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
13,Pedernales,DO,79.54,broken clouds,18.0384,-71.7440,Hotel Costa Marina
21,Kapaa,US,80.58,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
26,Tosham,IN,75.60,clear sky,28.8833,75.9167,Amrit home's
34,Acarau,BR,78.57,broken clouds,-2.8856,-40.1200,castelo pizzaria


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))